In [1]:
import json
from public_data.models import Ocsge
from django.db import connection

from public_data.models import CommunesSybarval

qs = CommunesSybarval.objects.all()
comm = qs[3]
bbox = comm.mpoly.extent

# Initial parameters
# bbox = [-0.601487594874066, 44.6932226204681, -1.02920400564268, 44.4183678096998]



# generate initial SQL
query = (
    "SELECT id, couverture_label, usage_label, millesime, map_color, year, st_AsGeoJSON(mpoly, 4) AS geojson " 
    f"FROM {Ocsge._meta.db_table} "
    "WHERE mpoly && ST_MakeEnvelope(%s, %s, %s, %s, 4326) "
    "LIMIT 2"
)

query

'SELECT id, couverture_label, usage_label, millesime, map_color, year, st_AsGeoJSON(mpoly, 4) AS geojson FROM public_data_ocsge WHERE mpoly && ST_MakeEnvelope(%s, %s, %s, %s, 4326) LIMIT 2'

In [33]:
with connection.cursor() as cursor:
    cursor.execute(query, bbox)
    rows = cursor.fetchall()
rows

[(138353,
  'Zones non bâties (Routes; places; parking…)',
  'Routier',
  datetime.date(2018, 1, 1),
  None,
  2018,
  '{"type":"MultiPolygon","coordinates":[[[[-1.2022,44.6398],[-1.2022,44.6398],[-1.2022,44.6396],[-1.2023,44.6395],[-1.2023,44.6395],[-1.2023,44.6394],[-1.2023,44.6394],[-1.2023,44.6394],[-1.2023,44.6394],[-1.2023,44.6394],[-1.2023,44.6394],[-1.2023,44.6393],[-1.2023,44.6393],[-1.2023,44.6393],[-1.2024,44.6394],[-1.2024,44.6394],[-1.2024,44.6394],[-1.2024,44.6394],[-1.2025,44.6394],[-1.2025,44.6394],[-1.2025,44.6394],[-1.2025,44.6394],[-1.2025,44.6394],[-1.2026,44.6394],[-1.2027,44.6394],[-1.2027,44.6394],[-1.2028,44.6394],[-1.2028,44.6394],[-1.2028,44.6394],[-1.2028,44.6394],[-1.2028,44.6394],[-1.2028,44.6394],[-1.2029,44.6394],[-1.203,44.6394],[-1.203,44.6394],[-1.203,44.6395],[-1.203,44.6395],[-1.2031,44.6395],[-1.2032,44.6395],[-1.2032,44.6395],[-1.2032,44.6394],[-1.2032,44.6394],[-1.2033,44.6394],[-1.2033,44.6394],[-1.2034,44.6394],[-1.2034,44.6394],[-1.2035,44.6394

In [35]:
# generate features
features = []
for row in rows:
    feature = {
        'type': 'Feature',
        'properties': {
            'id': row[0],
            'couverture_label': row[1],
            'usage_label': row[2],
            'millesime': row[3].strftime("%Y-%m-%d"),
            'map_color': row[4],
            'year': row[5],
        },
        'geometry': json.loads(row[6])
    }
    features.append(feature)
features

[{'type': 'Feature',
  'properties': {'id': 138353,
   'couverture_label': 'Zones non bâties (Routes; places; parking…)',
   'usage_label': 'Routier',
   'millesime': '2018-01-01',
   'map_color': None,
   'year': 2018},
  'geometry': {'type': 'MultiPolygon',
   'coordinates': [[[[-1.2022, 44.6398],
      [-1.2022, 44.6398],
      [-1.2022, 44.6396],
      [-1.2023, 44.6395],
      [-1.2023, 44.6395],
      [-1.2023, 44.6394],
      [-1.2023, 44.6394],
      [-1.2023, 44.6394],
      [-1.2023, 44.6394],
      [-1.2023, 44.6394],
      [-1.2023, 44.6394],
      [-1.2023, 44.6393],
      [-1.2023, 44.6393],
      [-1.2023, 44.6393],
      [-1.2024, 44.6394],
      [-1.2024, 44.6394],
      [-1.2024, 44.6394],
      [-1.2024, 44.6394],
      [-1.2025, 44.6394],
      [-1.2025, 44.6394],
      [-1.2025, 44.6394],
      [-1.2025, 44.6394],
      [-1.2025, 44.6394],
      [-1.2026, 44.6394],
      [-1.2027, 44.6394],
      [-1.2027, 44.6394],
      [-1.2028, 44.6394],
      [-1.2028, 44.6394

In [11]:
# convert python dictionary into json
geojson = json.dumps({
    "type": "FeatureCollection",
    "crs": {"type": "name", "properties": {"name": f"EPSG:4326"}},
    'features': features
})

print(geojson)

{"type": "FeatureCollection", "crs": {"type": "name", "properties": {"name": "EPSG:4326"}}, "features": [{"type": "Feature", "properties": {"id": 76772, "couverture_label": "Zones non b\u00e2ties (Routes; places; parking\u2026)", "usage_label": "Routier", "millesime": "2018-01-01", "map_color": null, "year": 2018}, "geometry": {"type": "MultiPolygon", "coordinates": [[[[-0.9517, 44.6414], [-0.9517, 44.6414], [-0.9517, 44.6414], [-0.9516, 44.6414], [-0.9516, 44.6414], [-0.9516, 44.6415], [-0.9516, 44.6414], [-0.9516, 44.6414], [-0.9517, 44.6414], [-0.9517, 44.6414], [-0.9517, 44.6414], [-0.9517, 44.6414], [-0.9517, 44.6414]]]]}}, {"type": "Feature", "properties": {"id": 76816, "couverture_label": "Zones non b\u00e2ties (Routes; places; parking\u2026)", "usage_label": "Routier", "millesime": "2018-01-01", "map_color": null, "year": 2018}, "geometry": {"type": "MultiPolygon", "coordinates": [[[[-0.8832, 44.485], [-0.8833, 44.485], [-0.8833, 44.4848], [-0.8833, 44.4848], [-0.8833, 44.4848]

In [28]:


qs = CommunesSybarval.objects.all()
qs = qs.annotate(extent=Extent("mpoly"))
comm = qs.first()
print(comm.extent)
print(comm.mpoly.extent)

bbox = [-0.601487594874066, 44.6932226204681, -1.02920400564268, 44.4183678096998]

bbox = [comm.extent[2], comm.extent[3], comm.extent[0], comm.extent[1]]
print(bbox)

(-0.919067041179307, 44.4183677659233, -0.678639609235308, 44.5734589145821)
(-0.919067041179307, 44.4183677659233, -0.678639609235308, 44.5734589145821)
[-0.678639609235308, 44.5734589145821, -0.919067041179307, 44.4183677659233]


In [11]:
# test avec jointure sur UsageSol
params = [-1.2747573852539065, 44.609534650700205, -1.0524559020996096, 44.69062362134582, 2015]

query = (
    "SELECT o.id, o.couverture_label, o.usage_label, o.millesime, u.map_color, "
    "o.year, st_AsGeoJSON(o.mpoly, 4) AS geojson "
    f"FROM {Ocsge._meta.db_table} o "
    f"INNER JOIN {UsageSol._meta.db_table} u ON u.code_prefix = o.usage "
    "WHERE o. mpoly && ST_MakeEnvelope(%s, %s, %s, %s, 4326) "
    "AND o.year = %s"
)

print(query)

with connection.cursor() as cursor:
    cursor.execute(query, params)
    rows = cursor.fetchall()

rows[:5]

SELECT o.id, o.couverture_label, o.usage_label, o.millesime, u.map_color, o.year, st_AsGeoJSON(o.mpoly, 4) AS geojson FROM public_data_ocsge o INNER JOIN public_data_usagesol u ON u.code_prefix = o.usage WHERE o. mpoly && ST_MakeEnvelope(%s, %s, %s, %s, 4326) AND o.year = %s


[(1499,
  'Zones à matériaux minéraux',
  'Ferré',
  datetime.date(2015, 1, 1),
  '#5a5a5a',
  2015,
  '{"type":"MultiPolygon","coordinates":[[[[-1.0518,44.6435],[-1.0514,44.6434],[-1.0509,44.6434],[-1.0502,44.6434],[-1.0501,44.6433],[-1.0498,44.6433],[-1.0496,44.6433],[-1.0494,44.6433],[-1.0494,44.6433],[-1.0491,44.6433],[-1.0491,44.6433],[-1.0488,44.6432],[-1.0488,44.6432],[-1.0485,44.6432],[-1.0485,44.6432],[-1.0484,44.6432],[-1.0484,44.6432],[-1.0478,44.6431],[-1.0477,44.6431],[-1.0476,44.6431],[-1.0473,44.643],[-1.0473,44.643],[-1.0473,44.643],[-1.0472,44.643],[-1.047,44.643],[-1.0469,44.6429],[-1.0468,44.6429],[-1.0462,44.6428],[-1.0462,44.6428],[-1.0461,44.6428],[-1.0461,44.6428],[-1.0457,44.6427],[-1.0456,44.6426],[-1.0456,44.6426],[-1.0455,44.6426],[-1.0454,44.6426],[-1.0453,44.6426],[-1.0453,44.6426],[-1.0446,44.6424],[-1.0446,44.6424],[-1.0444,44.6424],[-1.0442,44.6423],[-1.0439,44.6422],[-1.0438,44.6422],[-1.0437,44.6422],[-1.0437,44.6422],[-1.0437,44.6422],[-1.0437,44.6422

In [4]:
from public_data.models import RefPlan
table_name = RefPlan._meta.db_table
query = (
    "SELECT region_id, region_name, "
    "st_AsGeoJSON(ST_Union(mpoly), 8) AS geojson "
    f"FROM {table_name} "
    "GROUP BY region_id, region_name"
)
query

'SELECT region_id, region_name, st_AsGeoJSON(ST_Union(mpoly), 8) AS geojson FROM public_data_refplan GROUP BY region_id, region_name'

In [6]:
with connection.cursor() as cursor:
    cursor.execute(query)
    rows = cursor.fetchall()
len(rows)

13